# **World Development Data Analytics EDA using SQL**
## Importing necessary libraries & downloading our Dataset

In [1]:
import sqlite3 as sql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

{}

In [4]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

! chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [5]:
! kaggle datasets download -d psycon/world-development-indicators

 93% 296M/319M [00:01<00:00, 199MB/s]
100% 319M/319M [00:01<00:00, 181MB/s]


In [6]:
! unzip world-development-indicators.zip && rm world-development-indicators.zip

Archive:  world-development-indicators.zip
replace WDICountry-Series.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

## GDP EDA using SQL
Using SQL queries to explore this dataset

In [7]:
conn = sql.connect('/content/indicators.sqlite') #Connecting

In [8]:
pd.read_sql('SELECT * FROM sqlite_master WHERE type = "table";', conn)

,type,name,tbl_name,rootpage,sql
0,table,Country,Country,2,"CREATE TABLE Country (\n CountryCode TEXT,\..."
1,table,CountryNotes,CountryNotes,186,CREATE TABLE CountryNotes (\n Countrycode T...
2,table,Series,Series,948,"CREATE TABLE Series (\n SeriesCode TEXT,\n ..."
3,table,Indicators,Indicators,4448,CREATE TABLE Indicators (\n CountryName TEX...
4,table,SeriesNotes,SeriesNotes,1317550,CREATE TABLE SeriesNotes (\n Seriescode TEX...
5,table,Footnotes,Footnotes,1317587,CREATE TABLE Footnotes (\n Countrycode TEXT...


In [9]:
#Head, limit 10
pd.read_sql('SELECT * FROM Indicators LIMIT 11;', conn)

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Arab World,ARB,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,1.335609e+02
1,Arab World,ARB,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,8.779760e+01
2,Arab World,ARB,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,6.634579e+00
3,Arab World,ARB,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,8.102333e+01
4,Arab World,ARB,Arms exports (SIPRI trend indicator values),MS.MIL.XPRT.KD,1960,3.000000e+06
5,Arab World,ARB,Arms imports (SIPRI trend indicator values),MS.MIL.MPRT.KD,1960,5.380000e+08
6,Arab World,ARB,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,1960,4.769789e+01
7,Arab World,ARB,CO2 emissions (kt),EN.ATM.CO2E.KT,1960,5.956399e+04
8,Arab World,ARB,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,1960,6.439635e-01
9,Arab World,ARB,CO2 emissions from gaseous fuel consumption (%...,EN.ATM.CO2E.GF.ZS,1960,5.041292e+00


In [10]:
#Tail, limit 10
pd.read_sql('SELECT * FROM Indicators ORDER BY CountryName DESC Limit 11;', conn)

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Zimbabwe,ZWE,Total tax rate (% of commercial profits),IC.TAX.TOTL.CP.ZS,2015,32.8
1,Zimbabwe,ZWE,Time to resolve insolvency (years),IC.ISV.DURS,2015,3.3
2,Zimbabwe,ZWE,Time to prepare and pay taxes (hours),IC.TAX.DURS,2015,242.0
3,Zimbabwe,ZWE,Time required to start a business (days),IC.REG.DURS,2015,90.0
4,Zimbabwe,ZWE,Time required to register property (days),IC.PRP.DURS,2015,36.0
5,Zimbabwe,ZWE,Time required to get electricity (days),IC.ELC.TIME,2015,106.0
6,Zimbabwe,ZWE,Time required to enforce a contract (days),IC.LGL.DURS,2015,410.0
7,Zimbabwe,ZWE,Time required to build a warehouse (days),IC.WRH.DURS,2015,448.0
8,Zimbabwe,ZWE,Tax payments (number),IC.TAX.PAYM,2015,49.0
9,Zimbabwe,ZWE,Strength of legal rights index (0=weak to 12=s...,IC.LGL.CRED.XQ,2015,5.0


In [11]:
#Row Count
pd.read_sql('SELECT COUNT(*) AS row_count FROM Indicators;', conn)

,row_count
0,5656458


In [12]:
#Getting distinct values for CountryName

pd.read_sql('SELECT DISTINCT CountryName FROM Indicators;', conn)

,CountryName
0,Afghanistan
1,Albania
2,Algeria
3,American Samoa
4,Andorra
...,...
242,West Bank and Gaza
243,World
244,"Yemen, Rep."
245,Zambia


In [13]:
#Grouping Countries by Region
pd.read_sql('SELECT Region, COUNT(*) AS [Count] FROM Country GROUP BY Region ORDER BY 2 DESC;', conn)

,Region,Count
0,Europe & Central Asia,57
1,Sub-Saharan Africa,48
2,Latin America & Caribbean,41
3,East Asia & Pacific,36
4,,33
5,Middle East & North Africa,21
6,South Asia,8
7,North America,3


In [14]:
#Checking for NULL values
pd.read_sql("""SELECT COUNT(*) - COUNT(CountryName) AS CountryName,
                      COUNT(*) - COUNT(CountryCode) AS CountryCode,
                      COUNT(*) - COUNT(IndicatorName) AS IndicatorName,
                      COUNT(*) - COUNT(IndicatorCode) AS IndicatorCode,
                      COUNT(*) - COUNT(Year) AS Year,
                      COUNT(*) - COUNT(Value) AS Value
                FROM Indicators;""",
           conn)

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,0,0,0,0,0,0


In [49]:
#Checking for NULL values pt 2
pd.read_sql('SELECT COUNT(*) AS NaN FROM Indicators WHERE IndicatorName IS NULL;',conn)

,NaN
0,0


In [20]:
#Selecting Distinct Indicator Name
pd.read_sql('SELECT DISTINCT IndicatorName from Indicators;', conn)

,IndicatorName
0,"2005 PPP conversion factor, GDP (LCU per inter..."
1,"2005 PPP conversion factor, private consumptio..."
2,ARI treatment (% of children under 5 taken to ...
3,Access to electricity (% of population)
4,"Access to electricity, rural (% of rural popul..."
...,...
1339,Women's share of population ages 15+ living wi...
1340,"Youth literacy rate, population 15-24 years, b..."
1341,"Youth literacy rate, population 15-24 years, f..."
1342,"Youth literacy rate, population 15-24 years, g..."


## SQL Queries to explore GDP

In [21]:
#Let's take a look at GDP Indicators
pd.read_sql('Select DISTINCT IndicatorName FROM Indicators WHERE IndicatorName LIKE "GDP%";', conn)

,IndicatorName
0,GDP (constant LCU)
1,GDP (current LCU)
2,GDP at market prices (constant 2005 US$)
3,GDP at market prices (current US$)
4,GDP deflator (base year varies by country)
5,GDP growth (annual %)
6,GDP per capita (constant 2005 US$)
7,GDP per capita (constant LCU)
8,GDP per capita (current LCU)
9,GDP per capita (current US$)


In [30]:
#GDP continued
#Annual GDP Growth Rate for the U.S; Notice how GDP growth rate is negative for recesssion years, 2008-09
pd.read_sql("""SELECT CountryName, IndicatorName, Year, Value
               FROM Indicators WHERE IndicatorName = "GDP growth (annual %)"
               AND CountryName = "United States"
               AND Year >= 2007;""", conn)

,CountryName,IndicatorName,Year,Value
0,United States,GDP growth (annual %),2007,1.778570
1,United States,GDP growth (annual %),2008,-0.291621
2,United States,GDP growth (annual %),2009,-2.775530
3,United States,GDP growth (annual %),2010,2.531921
4,United States,GDP growth (annual %),2011,1.601455
5,United States,GDP growth (annual %),2012,2.321084
6,United States,GDP growth (annual %),2013,2.219308
7,United States,GDP growth (annual %),2014,2.388200


In [29]:
#GDP Continued
#Let's Compare U.S vs China's GDP Growth Rate

pd.read_sql("""SELECT CountryName, IndicatorName, Year, Value
               FROM Indicators
               WHERE IndicatorName= "GDP growth (annual %)"
               AND (CountryName= 'United States' OR CountryName= 'China')
               AND Year>=2007;""",
           conn)

,CountryName,IndicatorName,Year,Value
0,China,GDP growth (annual %),2007,14.194962
1,United States,GDP growth (annual %),2007,1.778570
2,China,GDP growth (annual %),2008,9.623377
3,United States,GDP growth (annual %),2008,-0.291621
4,China,GDP growth (annual %),2009,9.233551
5,United States,GDP growth (annual %),2009,-2.775530
6,China,GDP growth (annual %),2010,10.631708
7,United States,GDP growth (annual %),2010,2.531921
8,China,GDP growth (annual %),2011,9.484506
9,United States,GDP growth (annual %),2011,1.601455


In [33]:
#My mother is Guatemalan & my father is Cuban, so let's compare the 2 countries
pd.read_sql("""SELECT CountryName, IndicatorName, Year, Value
               FROM Indicators
               WHERE IndicatorName='GDP per capita (current US$)'
               AND (CountryName= 'Guatemala' OR CountryName= 'Cuba')
               AND Year>=2007;""",
           conn)

,CountryName,IndicatorName,Year,Value
0,Cuba,GDP per capita (current US$),2007,5193.484286
1,Guatemala,GDP per capita (current US$),2007,2472.389127
2,Cuba,GDP per capita (current US$),2008,5385.740727
3,Guatemala,GDP per capita (current US$),2008,2774.318060
4,Cuba,GDP per capita (current US$),2009,5494.925311
5,Guatemala,GDP per capita (current US$),2009,2617.112192
6,Cuba,GDP per capita (current US$),2010,5688.666732
7,Guatemala,GDP per capita (current US$),2010,2805.951392
8,Cuba,GDP per capita (current US$),2011,6092.613902
9,Guatemala,GDP per capita (current US$),2011,3166.582523


In [35]:
#Cont, GDP Growth Rate
pd.read_sql("""SELECT CountryName, IndicatorName, Year, Value
               FROM Indicators
               WHERE IndicatorName= "GDP growth (annual %)"
               AND (CountryName= 'Guatemala' OR CountryName= 'Cuba')
               AND Year>=2007;""",
           conn)


,CountryName,IndicatorName,Year,Value
0,Cuba,GDP growth (annual %),2007,7.262137
1,Guatemala,GDP growth (annual %),2007,6.304057
2,Cuba,GDP growth (annual %),2008,4.116828
3,Guatemala,GDP growth (annual %),2008,3.281080
4,Cuba,GDP growth (annual %),2009,1.449117
5,Guatemala,GDP growth (annual %),2009,0.526043
6,Cuba,GDP growth (annual %),2010,2.388333
7,Guatemala,GDP growth (annual %),2010,2.869487
8,Cuba,GDP growth (annual %),2011,2.802964
9,Guatemala,GDP growth (annual %),2011,4.162049


In [37]:
#Highest GDP per Capita in Entire Dataset
pd.read_sql("""SELECT CountryName, Year, MAX (Value)
               FROM Indicators
               WHERE IndicatorName = 'GDP per capita (current US$)'
               ;""",
            conn)

,CountryName,Year,MAX (Value)
0,Monaco,2008,193648.132213


In [39]:
#Lowest GDP Per Capita in Entire Dataset
pd.read_sql("""SELECT CountryName, Year, MIN (Value)
               FROM Indicators
               WHERE IndicatorName = 'GDP per capita (current US$)'
               ;""",
            conn)

,CountryName,Year,MIN (Value)
0,Equatorial Guinea,1962,35.367727


In [47]:
#Average GDP Per Capita For Each Country
pd.read_sql(""" SELECT CountryName, AVG(Value)
                FROM Indicators
                WHERE IndicatorName= 'GDP per capita (current US$)'
                GROUP BY CountryName;""",
           conn)

,CountryName,AVG(Value)
0,Afghanistan,253.948534
1,Albania,1893.266623
2,Algeria,1964.687652
3,Andorra,19981.425118
4,Angola,1736.088672
...,...,...
234,West Bank and Gaza,1752.297159
235,World,4019.677734
236,"Yemen, Rep.",738.636427
237,Zambia,583.761921


In [38]:
#Highest GDP Annual Growth Rate in Entire Dataset
pd.read_sql("""SELECT CountryName, Year, MAX (Value)
               FROM Indicators
               WHERE IndicatorName = 'GDP growth (annual %)'
               ;""",
            conn)

,CountryName,Year,MAX (Value)
0,Oman,1965,189.829926


In [46]:
#Lowest GDP Annual Growth Rate in Entire Dataset
pd.read_sql("""SELECT CountryName,Year, MIN (Value)
               FROM Indicators
               WHERE IndicatorName = 'GDP growth (annual %)'
               ;""",
            conn)

,CountryName,Year,MIN (Value)
0,Iraq,1991,-64.047107


In [44]:
#Average GDP Annual Growth Rate For Each Country
pd.read_sql(""" SELECT CountryName, AVG(Value)
                FROM Indicators
                WHERE IndicatorName= 'GDP growth (annual %)'
                GROUP BY CountryName;""",
           conn)

,CountryName,AVG(Value)
0,Afghanistan,8.071227
1,Albania,2.894555
2,Algeria,3.835124
3,Andorra,2.710765
4,Angola,2.805847
...,...,...
233,West Bank and Gaza,5.927586
234,World,3.464413
235,"Yemen, Rep.",3.795132
236,Zambia,3.374269
